In [1]:
import keplergl as kp
import geopandas as gpd
import pandas as pd
import json
import shapely.geometry
import pyproj

In [2]:
# Dataframe with the location of the points
stops = pd.read_csv(r"C:\Users\santi\Google Drive\Master\Python\Kepler-GL\data\ov_ridership\stops.txt")

# Dataframe with the data at the point we want to show
# In this case boardings and alightings by stop
ridership = pd.read_csv(r"C:\Users\santi\Google Drive\Master\Python\Kepler-GL\data\ov_ridership\original_data.csv")

# Make sure the format of the stop_id in both datasets match
ridership['boarding stop'] = ridership['boarding stop'].map(str)

# Filter only the stops that have ridership data 
# This will speed-up the script
filtered = stops.loc[stops.stop_id.isin(ridership['boarding stop'].unique())]

# Create a GeoDataFrame with a Point geometry
stops = gpd.GeoDataFrame(data=filtered, geometry = gpd.points_from_xy(filtered['stop_lon'], filtered['stop_lat']))
stops.crs={'init':'epsg:4326'}

In [4]:
# Change the GeoDataFrame to the right projection in order to use meters

import utm

stops.reset_index(inplace=True)

lat = stops.geometry[0].y
lon = stops.geometry[0].x

zone = utm.from_latlon(lat, lon)
zone

def code(zone):
    #The EPSG code is 32600+zone for positive latitudes and 32700+zone for negatives.
    if lat <0:
        epsg_code = 32700 + zone[2]
    else:
        epsg_code = 32600 + zone[2]
    return epsg_code

stops_proj = stops.to_crs(epsg=code(zone))

C:\Users\santi\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [5]:
# Create corners of the big rectangle to be transformed to a grid
sw = shapely.geometry.Point((stops_proj.geometry.x.min(), stops_proj.geometry.y.min()))
ne = shapely.geometry.Point((stops_proj.geometry.x.max(), stops_proj.geometry.y.max()))

stepsize = 100 # 100 m grid step size

# Iterate over 2D area
gridpoints = []
x = sw.x
while x < ne.x:
    y = sw.y
    while y < ne.y:
        # Get the parameters to create the 100 m x 100 m  box
        minx = x - stepsize/2
        miny = y - stepsize/2
        maxx = x + stepsize/2
        maxy = y + stepsize/2
        
        # Create the box
        box = shapely.geometry.box(minx, miny, maxx, maxy) #minx, miny, maxx, maxy) 
        gridpoints.append(box)
        y += stepsize
    x += stepsize

In [6]:
# Create a GeoDataFrame with the grid created above
grid_id = list(range(0, len(gridpoints)))
gdf = gpd.GeoDataFrame(data = grid_id, geometry=gridpoints)
gdf.columns = ['index', 'geometry']

# Change the projection to EPSG:4326
gdf.crs = {'init':'epsg:' + str(code(zone))}
gdf = gdf.to_crs(epsg=4326)
gdf.head(1)

,0,geometry
0,0,"POLYGON ((206751.443 5802266.462, 206751.443 5..."


In [42]:
# Keep only the important data of the stops to speed up the script
stops1 = stops.loc[:,['stop_code', 'stop_id', 'geometry']]

# Look within which box of the grid each stop is
joined = gpd.sjoin(stops1, gdf , how='left', op='within') 
joined.head()

,stop_code,stop_id,geometry,index_right,index
0,1331917,10154220,POINT (6.58959 53.26352),1610895.0,1610895.0
1,1332649,12102410,POINT (6.68485 53.10425),1681186.0,1681186.0
2,1330678,12155820,POINT (6.66472 53.03665),1661671.0,1661671.0
3,656630,10274120,POINT (6.51140 53.33436),1551440.0,1551440.0
4,1332259,13012510,POINT (6.92699 53.13879),1879264.0,1879264.0


In [49]:
# Make sure the key columns have the same format
ridership['boarding stop code'] = ridership['boarding stop code'].map(str)

# Join Dataframes in order to have the box grid index and number of
# boardings in the same Dataframe
# Here we look for the origin_area_id
start = pd.merge(ridership, joined, left_on='boarding stop code', right_on='stop_id', how='left')
start.columns = ['Date', 'boarding stop', 'boarding stop code', 'alignment',
       'alignment stop code', 'boarding weekday', 'boarding Saturday',
       'boarding Sunday', 'stop_code', 'stop_id', 'geometry', 'origin_area_id',
       'index']

# Here we look for the destination_area_id
start['alignment stop code'] = start['alignment stop code'].map(str)
destination = pd.merge(start, joined, left_on='alignment stop code', right_on = 'stop_id', how='left')
destination = destination.loc[:,['origin_area_id', 'index_right','boarding weekday']]
destination.columns = ['origin_area_id', 'destination_area_id', 'count']

# Create the matrix Dataframe
matrix = destination.pivot_table('count', index=['origin_area_id', 'destination_area_id'], aggfunc='sum').reset_index()
matrix.head()

,Date,boarding stop,boarding stop code,alignment,alignment stop code,boarding weekday,boarding Saturday,boarding Sunday,stop_code,stop_id,geometry,origin_area_id,index
0,11/2/2020,"Groningen, Hoofdstation",10009013,"Groningen, Zernikeplein",10002010,1779,0,0,1331461,10009013,POINT (6.56633 53.21130),1590182.0,1590182.0
1,11/2/2020,"Groningen, Zernikeplein",10002000,"Groningen, Hoofdstation",10009013,1656,0,0,1332117,10002000,POINT (6.53399 53.24104),1564701.0,1564701.0
2,11/2/2020,"Groningen, Zernikeplein",10002000,"Groningen, Hoofdstation, uitstaphalte",10009000,887,0,0,1332117,10002000,POINT (6.53399 53.24104),1564701.0,1564701.0
3,11/2/2020,"Groningen, Hoofdstation",10009013,"Groningen, Nijenborgh",10004140,869,0,0,1331461,10009013,POINT (6.56633 53.21130),1590182.0,1590182.0
4,11/2/2020,"Groningen, Hoofdstation",10009019,"Groningen, Grote Markt",10006280,694,0,0,1331960,10009019,POINT (6.56782 53.21166),1591398.0,1591398.0


In [108]:
# Get the origin_area_id for all the boxes with boardings data
all_ids = matrix['origin_area_id'].unique()

# Get the destination_area_id for all the boxes with alightings data
more_ids = matrix.loc[~matrix.destination_area_id.isin(matrix.origin_area_id.unique()),'destination_area_id'].unique()

# Join the last two in one list
all_ids = all_ids.append(more_ids)

# Use the list with all the ids to filter only the part of the 
# grid that has data associated
filtered_grid = gdf.loc[gdf.index.isin(all_ids),:]

In [115]:
# Save the grid as a shapefile
prj = 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]'

filtered_grid.to_file(driver = 'ESRI Shapefile',
                filename = r"C:\Users\santi\Google Drive\Master\Python\Kepler-GL\grid\filtered_grid.shp",
                encoding='utf-8',
                crs_wkt=prj)

# Save the matrix as a csv
matrix.to_csv(r"C:\Users\santi\Google Drive\Master\Python\Kepler-GL\grid\matrix.csv", index=False)

In [117]:
# Preview on a map
m = kp.KeplerGl(height=600, data={'data':filtered_grid})
m.add_data(data=stops1, name='stops')
m

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'data':            index                                           geometry
0              0  P…